# Generative Agents in LangChain

This notebook implements a generative agent based on the paper [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442) by Park, et. al.

In it, we leverage a time-weighted Memory object backed by a LangChain Retriever.

# Setup

In [1]:
# Use termcolor to make it easy to colorize the outputs.
!pip install termcolor > /dev/null

The system cannot find the path specified.


In [2]:
import logging

logging.basicConfig(level=logging.ERROR)

In [3]:
from datetime import datetime, timedelta
from typing import List

from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from termcolor import colored

In [4]:
USER_NAME = "Bryan Chan"  # The name you want to use when interviewing the agent.
LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.

# Create Memory Framework

### Generative Agent Memory Components


This tutorial highlights the memory of generative agents and its impact on their behavior. The memory varies from standard LangChain Chat memory in two aspects:

1. **Memory Formation**

   Generative Agents have extended memories, stored in a single stream:
      1. Observations - from dialogues or interactions with the virtual world, about self or others
      2. Reflections - resurfaced and summarized core memories


2. **Memory Recall**

   Memories are retrieved using a weighted sum of salience, recency, and importance.



You can review the definitions of the functions & keyword arguments in the reference documentation:

- [GenerativeAgentMemory](https://python.langchain.com/api_reference/experimental/generative_agents/langchain_experimental.generative_agents.memory.GenerativeAgentMemory.html): **Memory** for the generative agent 
   - `llm`
   - `memory_retriever` = create_new_memory_retriever()
   - `current_plan`
   - `reflection_threshold`
   - `add_memory`

- [GenerativeAgent](https://python.langchain.com/api_reference/experimental/generative_agents.html): Agent as a character with **memory** and innate **characteristics**,  
   - basics like `name`, `age` and `llm`
   - `memory` object that combines relevance, recency, and ‘importance’
   - `summary` and `summary_refresh_seconds` to set how frequently to re-generate the summary
   - `summarize_related_memories`: Summarize memories that are most relevant to an observation
   - `status` traits of the character you wish not to change
   - `traits` set Permanent traits to ascribe to the character 
   - `generate_dialogue_response`


In [5]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

In [6]:
# View generative_agents library documentation
help(GenerativeAgent)

Help on class GenerativeAgent in module langchain_experimental.generative_agents.generative_agent:

class GenerativeAgent(pydantic.main.BaseModel)
 |  GenerativeAgent(*, name: str, age: Optional[int] = None, traits: str = 'N/A', status: str, memory: langchain_experimental.generative_agents.memory.GenerativeAgentMemory, llm: langchain_core.language_models.base.BaseLanguageModel, verbose: bool = False, summary: str = '', summary_refresh_seconds: int = 3600, last_refreshed: datetime.datetime = None, daily_summaries: List[str] = None) -> None
 |
 |  Agent as a character with memory and innate characteristics.
 |
 |  Method resolution order:
 |      GenerativeAgent
 |      pydantic.main.BaseModel
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  chain(self, prompt: langchain_core.prompts.prompt.PromptTemplate) -> langchain.chains.llm.LLMChain
 |      Create a chain with the same settings as the agent.
 |
 |  generate_dialogue_response(self, observation: str, now: Optional[datetime

## Memory Lifecycle

Summarizing the key methods in the above: `add_memory` and `summarize_related_memories`.

When an agent ***makes* an observation**, it stores the memory:
    
1. Language model scores the memory's importance (1 for mundane - 10 for poignant)
2. Observation and importance are stored within a document by TimeWeightedVectorStoreRetriever, with a `last_accessed_time`.

When an agent ***responds* to an observation**:

1. Generates query(s) for retriever, which fetches documents based on salience + recency + importance.
2. Summarizes the retrieved information
3. Updates the `last_accessed_time` for the used documents.


## Create a Generative Character

Now that we've walked through the definition, we will create two characters named "Tommie" and "Eve".

We use the `faiss` (**Facebook AI Similarity Search**) library to quickly search for embeddings of multimedia documents that are similar to each other, particularly useful for tasks involving large-scale nearest neighbor search

In [7]:
# Import Libraries
import math
import faiss #for efficient similarity search and clustering of dense vectors


In [8]:
# Relevance Score function
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""

    return 1.0 - score / math.sqrt(2)

    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)

    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)


In [9]:
# Create <emory Retriever
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""

    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    
    # Initialize the vectorstore as empty
    embedding_size = 1536                           # set dimension of the embeddings (produced by the embedding model)
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),                       # initialize an empty Memory docstore
        {},                                         # index-to-document store ID mapping
        relevance_score_fn = relevance_score_fn,    # scoring the relevance of search results
    )

    # Time-weighted scoring mechanism
    return TimeWeightedVectorStoreRetriever(vectorstore=vectorstore,            
                                            other_score_keys=["importance"],    # add also the importance score
                                            k=15                                # return up to 15 relevant documents
    )

# Single Agent Setup

Using the functions of `GenerativeAgentMemory()` and `GenerativeAgent()` from [Langchain Library](https://python.langchain.com/api_reference/experimental/generative_agents.html)

In [10]:
# Memory Management system (storing, retrieval, reflection)
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever = create_new_memory_retriever(),
    verbose=False,
    reflection_threshold = 8,  # we will give this a relatively low number to show how reflection works
)


In [11]:
# Define the Agent Characteristics
# (name, age, traits, status, memory_retriever, llm, memory)
tommie = GenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for a job",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=tommies_memory, # Assign the memory management system to the agent
)

In [12]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is independent, confident, and straightforward.


In [13]:
# List of Additional Observations
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]

# Add the observations to the memory using the 'add_memory()' function
for observation in tommie_observations:
    tommie.memory.add_memory(observation) # loop through the observations and add to memory

In [14]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is nostalgic, hungry, tired, and trying to rest in a new home with noisy neighbors and a cat nearby.


## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [15]:
# Create agent to interview Tommie
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [16]:
interview_agent(tommie, "What do you like to do?")

'Tommie said "I really enjoy design and talking with people. What about you?"'

In [12]:
interview_agent(tommie, "What are you looking forward to doing today?")

'Tommie said "Well, I\'m actually looking for a job right now, so hopefully I can find some job postings online and start applying. How about you, Person A? What\'s on your schedule for today?"'

In [13]:
interview_agent(tommie, "What are you most worried about today?")

'Tommie said "Honestly, I\'m feeling pretty anxious about finding a job. It\'s been a bit of a struggle lately, but I\'m trying to stay positive and keep searching. How about you, Person A? What worries you?"'

## Run Simulation

In [17]:
# Add Additional Observations & Events
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
]

Run the simulation for a day. 
- Estimated duration: 17.5 minutes
- Cost: $0.03

In [18]:
# Run Simulation
# loops over a collection of observations, generating and printing reactions for each observation
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)

    # Print the observation and the agent's reaction
    print(colored(observation, "green"), reaction)

    # Every 20 observations, print a summary of the agent's state 
    # using the 'get_summary()' function
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

Tommie wakes up to the sound of a noisy construction site outside his window. Tommie covers his ears and tries to go back to sleep.
Tommie gets out of bed and heads to the kitchen to make himself some coffee. Tommie smiles and thinks to himself, "Coffee always helps me start the day right."
Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some. Tommie sighs in frustration and makes a mental note to buy coffee filters later.
Tommie finally finds the filters and makes himself a cup of coffee. Tommie takes a sip of his coffee and feels a sense of relief.
The coffee tastes bitter, and Tommie regrets not buying a better brand. Tommie grimaces at the bitter taste and makes a mental note to buy a better brand next time.
Tommie checks his email and sees that he has no job offers yet. Tommie feels a pang of disappointment but remains determined to keep looking for job opportunities.
Tommie spends some time updating his resume and cover letter

## Interview after the day

In [16]:
interview_agent(tommie, "Tell me about how your day has been going")

'Tommie said "It\'s been a bit of a rollercoaster, to be honest. I\'ve had some setbacks in my job search, but I also had some good moments today, like sending out a few resumes and meeting some potential employers at a job fair. How about you?"'

In [17]:
interview_agent(tommie, "How do you feel about coffee?")

'Tommie said "I really enjoy coffee, but sometimes I regret not buying a better brand. How about you?"'

In [18]:
interview_agent(tommie, "Tell me about your childhood dog!")

'Tommie said "Oh, I had a dog named Bruno when I was a kid. He was a golden retriever and my best friend. I have so many fond memories of him."'

# Multi Agent Setup

## Adding Second Character

Let's add a second character to have a conversation with Tommie, using the same functions of `GenerativeAgentMemory()` and `GenerativeAgent()` from [Langchain Library](https://python.langchain.com/api_reference/experimental/generative_agents.html)

In [1]:
# Memory Management system (storing, retrieval, reflection)
eves_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5,
)

NameError: name 'GenerativeAgentMemory' is not defined

In [ ]:
# Define the Agent Characteristics
eve = GenerativeAgent(
    name="Eve",
    age=34,
    traits="curious, helpful",  # You can add more persistent traits here
    status="N/A",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    daily_summaries=[
        (
            "Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie."
        )
    ],
    memory=eves_memory,
    verbose=False,
)

In [48]:
# Add Base Observations
yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")

eve_observations = [
    "Eve wakes up and hear's the alarm",
    "Eve eats a boal of porridge",
    "Eve helps a coworker on a task",
    "Eve plays tennis with her friend Xu before going to work",
    "Eve overhears her colleague say something about Tommie being hard to work with",
]

for observation in eve_observations:
    eve.memory.add_memory(observation)

# View the current observations in eve memory
eve.memory

In [49]:
print(eve.get_summary())

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is a helpful and active person who enjoys sports and takes care of her physical health. She is attentive to her surroundings, including her colleagues, and has good time management skills.


## Pre-conversation interviews


Let's "Interview" Eve before she speaks with Tommie.

In [50]:
interview_agent(eve, "How are you feeling about today?")

'Eve said "I\'m feeling pretty good, thanks for asking! Just trying to stay productive and make the most of the day. How about you?"'

In [51]:
interview_agent(eve, "What do you know about Tommie?")

'Eve said "I don\'t know much about Tommie, but I heard someone mention that they find them difficult to work with. Have you had any experiences working with Tommie?"'

In [52]:
interview_agent(
    eve,
    "Tommie is looking to find a job. What are are some things you'd like to ask him?",
)

'Eve said "That\'s interesting. I don\'t know much about Tommie\'s work experience, but I would probably ask about his strengths and areas for improvement. What about you?"'

In [53]:
interview_agent(
    eve,
    "You'll have to ask him. He may be a bit anxious, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.",
)

'Eve said "Sure, I can keep the conversation going and ask plenty of questions. I want to make sure Tommie feels comfortable and supported. Thanks for letting me know."'

## Run Simulation

 an infinite loop where each agent ***takes turns*** generating responses based on the latest observation

In [54]:
def run_conversation(agents: List[GenerativeAgent],             # get a list of agents
                     initial_observation: str) -> None:         # get the 1st observation
    """Runs a conversation between agents."""

    _, observation = agents[1].generate_reaction(initial_observation)   # generate a reaction to observation
    print(observation)

    turns = 0

    # Enters a loop where agents take turns generating responses
    while True:
        break_dialogue = False

        for agent in agents:
            
            # use 'generate_dialogue_response()' to generate 2 values: stay_in_dialogue and observation
            stay_in_dialogue, observation = agent.generate_dialogue_response(observation)
            print(observation)

            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True           # if agent wants to break the dialogue, set to True
                
        if break_dialogue:
            break
        turns += 1

In [ ]:
# Set the List of Agents
agents = [tommie, eve]

# Run the conversation
run_conversation(
    agents,
    "Tommie said: Hi, Eve. Thanks for agreeing to meet with me today. I have a bunch of questions and am not sure where to start. Maybe you could first share about your experience?",
)

## Let's interview our agents after their conversation

Since the generative agents retain their memories from the day, we can ask them about their plans, conversations, and other memoreis.

We can see a current "Summary" of a character based on their own perception of self has changed

In [56]:
# Print the Summary of the Tommie
# We can see a current "Summary" of a character based on their own perception of self has changed
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is determined and hopeful in his job search, but can also feel discouraged and frustrated at times. He has a strong connection to his childhood dog, Bruno. Tommie seeks support from his friends when feeling overwhelmed and is grateful for their help. He also enjoys exploring his new city.


In [57]:
# Print the summary of Eve
print(eve.get_summary(force_refresh=True))

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is a helpful and friendly person who enjoys playing sports and staying productive. She is attentive and responsive to others' needs, actively listening and asking questions to understand their perspectives. Eve has experience in event planning and communication, and is willing to share her knowledge and expertise with others. She values teamwork and collaboration, and strives to create a comfortable and supportive environment for everyone.


In [58]:
interview_agent(tommie, "How was your conversation with Eve?")

'Tommie said "It was really helpful actually. Eve shared some great tips on managing events and handling unexpected issues. I feel like I learned a lot from her experience."'

In [59]:
interview_agent(eve, "How was your conversation with Tommie?")

'Eve said "It was great, thanks for asking. Tommie was very receptive and had some great questions about event planning. How about you, have you had any interactions with Tommie?"'

In [60]:
interview_agent(eve, "What do you wish you would have said to Tommie?")

'Eve said "It was great meeting with you, Tommie. If you have any more questions or need any help in the future, don\'t hesitate to reach out to me. Have a great day!"'